### Configure Loopbacks From A List

One powerful feature of Netmiko is the ability to push configuration information. As previously discussed this process is also initutive for most network engineers because the format Netmiko expects configuration data is the same way it would be issued from a CLI.

In this lab we are going to push Loopback configuration to the two routers in the Programmability Lab Guide.

In order to utilize Netmiko we need to import it. There are two commonly used methods:
* ```from netmiko import Netmiko```
* ```from netmiko import ConnectHandler```

In [1]:
import yaml
from getpass import getpass
from netmiko import Netmiko
from pprint import pprint

### Read Inventory File

The source of truth for our network configuration will be a YAML file called ```routers.yml```:
```yaml
r1:
  host: 192.168.2.161
  port: 22
  device_type: cisco_ios
  eigrp:
    as: 1
    advertised_networks:
      - network: 1.1.1.1
        prefix: 0.0.0.0
      - network: 172.31.12.0
        prefix: 0.0.0.255
  interfaces:
    - name: loopback1
      address: 1.1.1.1
      netmask: 255.255.255.255
      enabled: true

r2:
  host: 192.168.2.162
  port: 22
  device_type: cisco_ios
  eigrp:
    as: 1
    advertised_networks:
      - network: 2.2.2.2
        prefix: 0.0.0.0
      - network: 172.31.12.0
        prefix: 0.0.0.255
  interfaces:
    - name: loopback2
      address: 2.2.2.2
      netmask: 255.255.255.255
      enabled: true
```

The ``` def read_inventory``` function will utilize the Python yaml library to read this file and return a dictionary.

In [2]:
def read_inventory(inventory_file):
    with open(inventory_file, 'r') as f:
        inventory = yaml.safe_load(f)
    return inventory

### Create a List

The Netmiko send_config_set method expects the configuration data in a list format. This is true even if you had a single command. For example if you wanted to set the ntp server to 192.168.100.5 you would need to format the data as:
```
config_command = ['ntp server 192.168.100.5']
```

The ```configure_loopback``` is a basic function that will take the interfaces information from our inventory file and return a list:
```yaml
interfaces:
  - name: loopback2
    address: 2.2.2.2
    netmask: 255.255.255.255
    enabled: true
```

will return:
```python
config_command = ['interface loopback2', 'ip address 2.2.2.2 255.255.255', 'no shutdown']
```

In [3]:
def configure_loopback(interfaces):
    for interface in interfaces:
        interface_configuration = list()
        interface_configuration.append(f"interface {interface['name']}")
        interface_configuration.append(f"ip address {interface['address']} {interface['netmask']}")
        if interface['enabled'] is True:
            interface_configuration.append('no shutdown')
        else:
            interface_configuration.append('shutdown')
    return interface_configuration

### Main Execution

First thing is read in the inventory file.

In [4]:
if __name__ == '__main__':
    username=input('Enter your username: ')
    password=getpass('Enter your password: ')

Enter your username: admin
Enter your password: ········


In [5]:
    inventory_file = 'inventory/routers.yml'
    inventory = read_inventory(inventory_file)

### Log into the routers and execute the commands

After reading in our inventory file we are going to loop through the inventory. 

The first task is call the ```configure_loopback``` method to create our configuration list.

Netmiko has two possible ways that you can create a session. The first option is to create a dictionary:
```python
some_device = {'device_type': 'cisco_xe',
        'host': '10.246.32.203',
        'port': 3222,
        'username': username,
        'password': password,
        'device_type': device type}
```
Supported device_types can be found on Netmiko's website.

After defining your dictionary you can start a context manager and pass the dictionary to Netmiko:
```python
with Netmiko(**some_device) as session:
    # Continue
```

Another option is detailed below and that is to pass the variables to Netmiko.

After connecting to the device you can utilize Netmiko's find_prompt() which will return the device's prompt. 

We will then utilize Netmiko's send_config_set method passing in our generated configuration list from the configure_loopback function.

We can also print out the output. This is how Netmiko issued our requested commands. This should look very familiar to network engineers.

In [6]:
    for key, value in inventory.items():
        interface_configuration = configure_loopback(value['interfaces'])
        print(f"Logging into: {key}")
        with Netmiko(host=value['host'], 
                 port=value['port'],
                 username=username, 
                 password=password, 
                 device_type=value['device_type']) as session:
            print(session.find_prompt())
            output = session.send_config_set(interface_configuration)
            print(output)
        

Logging into: csr1
CSR1#
config term
Enter configuration commands, one per line.  End with CNTL/Z.
CSR1(config)#interface loopback1
CSR1(config-if)#ip address 1.1.1.1 255.255.255.255
CSR1(config-if)#no shutdown
CSR1(config-if)#end
CSR1#
Logging into: csr3
CSR3#
config term
Enter configuration commands, one per line.  End with CNTL/Z.
CSR3(config)#interface loopback3
CSR3(config-if)#ip address 3.3.3.3 255.255.255.255
CSR3(config-if)#no shutdown
CSR3(config-if)#end
CSR3#
